In [55]:
import os
import pandas as pd
from datetime import datetime
import re
# SITE_BIRTH_FILE = "/www/site_birth.csv"
# SITE_TABLE = "/www/site_table.conf"

# 网站创建日期记录表(csv)是记录累加的(各个批次的总和),而conf文件是覆盖的
SITE_BIRTH_CSV = "./site_birth.csv"
SITE_TABLE_CONF = "./site_table.conf"
# 本批次建站列表备份文件(可选)
SITE_TABLE_BAK = "./site_table.conf.bak"

NGINX_VHOST_ROOT='/www/server/panel/vhost/nginx'

In [ ]:

# csv表头
TABLE_HEADER=["domain", "birth_time"]
now = datetime.now()

# 标准日期-时间格式 YYYY-MM-DD HH:MM:SS (时间(时分秒对于此任务不重要,可以仅仅保留日期(年月日)))
dt = now.strftime('%Y-%m-%d %H:%M:%S')

# 检查网站创建日期日志文件(csv)是否存在,如果不存在,则创建此文件,表头为domain,birth_time
if os.path.exists(SITE_BIRTH_CSV) == False:
    print("日志文件不存在,创建模板日志文件(csv)...")
    df = pd.DataFrame(columns=TABLE_HEADER)
    df.to_csv(SITE_BIRTH_CSV, index=False)
else:
    print("日志文件存在,读取站点创建日期...")
    df = pd.read_csv(SITE_BIRTH_CSV)


site_birth_lines = []
with open(SITE_TABLE_CONF, "r") as f:
    lines = f.readlines()
    # print(lines)
    for line in lines:
        if re.match(r"^\s*#", line):
            print(f"skip line:{line}")
        else:
            domain_match = re.match(r"^\s*(?:https?://w*\.?)?(?P<domain>.*?)\s+", line)
            # print(f"{domain_match.groupdict()}")
            domain = domain_match.groupdict()["domain"]
            # print(f"domain:{domain}")
            # 构造用于插入csv的字典
            site_dict = {"domain": domain, "birth_time": dt}
            site_birth_lines.append(site_dict)
            # print(f"site_dict:{site_dict}")
print(f"site_birth_lines:{site_birth_lines}")
df = pd.concat([df, pd.DataFrame(site_birth_lines)], ignore_index=True)
df.to_csv(SITE_BIRTH_CSV, index=False)
print(f"{df}")

In [ ]:

DAYS = 10  # 可根据需要调整

# 读取CSV文件
df = pd.read_csv(SITE_BIRTH_CSV)

# 将birth_time列转换为datetime类型
df['birth_time'] = pd.to_datetime(df['birth_time'])

# 设置阈值天数

# 获取当前时间
current_time = datetime.now()

# 过滤出距离当前时间不超过DAYS天的行
filtered_df = df[(current_time - df['birth_time']).dt.days <= DAYS]

print(filtered_df)

         domain          birth_time
1   domain3.com 2025-12-02 18:07:17
4  domain11.com 2025-12-12 18:09:22
5  domain13.com 2025-12-12 18:09:22
6  domain14.com 2025-12-12 18:09:22
7  domain15.com 2025-12-12 18:09:22


In [61]:
domains = filtered_df["domain"]
domains = list(domains)
vhost_confs=[]

In [ ]:
for domain in domains:
    # 构造路径
    path = os.path.join(NGINX_VHOST_ROOT, domain)+'.conf'
    # print(path)
    vhost_confs.append(path)
for config in vhost_confs:
    young2old(config)
    # print(config)